In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

# Initialization

In [11]:
L_min, L_max = 30e-6, 2000e-6
C_min, C_max = 20e-6, 1000e-6
fsw_min, fsw_max = 20e3, 200e3
t_dt_min, t_dt_max = 175.2e-9, 262.8e-9
d_cycle_min, d_cycle_max = 0.2, 0.35


# Path to the model directory
model_dir = 'D:/Krishna/ai-power-converter/new/ml/model/'

# List of model filenames
model_filenames = [
    'current_delta_current.h5',
    'delta_voltage.h5',
    'P_CAP_ESR.h5',
    'P_COSS.h5',
    'P_D.h5',
    'P_G.h5',
    'P_L_DCR.h5',
    'P_ON_H_P_ON_L.h5',
    'voltage.h5'
]


# Scaling

In [3]:
def custom_scaling(X):
    return np.array([(X[:, 0] - L_min) / (L_max - L_min), 
                     (X[:, 1] - C_min) / (C_max - C_min),
                     (X[:, 2] - fsw_min) / (fsw_max - fsw_min),
                     (X[:, 3] - t_dt_min) / (t_dt_max - t_dt_min),
                     (X[:, 4] - d_cycle_min) / (d_cycle_max - d_cycle_min)]).T

def inverse_custom_scaling(X_scaled):
    X_inverse = np.empty_like(X_scaled)
    X_inverse[:, 0] = X_scaled[:, 0] * (L_max - L_min) + L_min
    X_inverse[:, 1] = X_scaled[:, 1] * (C_max - C_min) + C_min
    X_inverse[:, 2] = X_scaled[:, 2] * (fsw_max - fsw_min) + fsw_min
    X_inverse[:, 3] = X_scaled[:, 3] * (t_dt_max - t_dt_min) + t_dt_min
    X_inverse[:, 4] = X_scaled[:, 4] * (d_cycle_max - d_cycle_min) + d_cycle_min
    return X_inverse

# Predict

In [18]:
input_array = np.array([[0.00003, 2.44021e-05, 20000, 175e-09,0.2]])

input_scaled = custom_scaling(input_array)
input_array_inv = inverse_custom_scaling(input_scaled)

print(input_array)
print(input_scaled)
print(input_array_inv)

# Load and test each model
models = {}
predictions = {}

for filename in model_filenames:
    # Construct the full path to the model file
    model_path = os.path.join(model_dir, filename)
    
    # Check if the file exists
    if os.path.exists(model_path):
        # Load the model
        model = load_model(model_path)
        models[filename] = model
        
        # Make a prediction
        prediction = model.predict(input_array)
        predictions[filename] = prediction
    else:
        print(f"File not found: {model_path}")

# Print the predictions
for filename, prediction in predictions.items():
    print(f"Prediction for {filename}: {prediction}")

[[3.00000e-05 2.44021e-05 2.00000e+04 1.75000e-07 2.00000e-01]]
[[ 0.          0.00449194  0.         -0.00228311  0.        ]]
[[3.00000e-05 2.44021e-05 2.00000e+04 1.75000e-07 2.00000e-01]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction for current_delta_current.h5: [[0. 0.]]
Prediction for delta_voltage.h5: [[0.]]
Prediction for P_CAP_ESR.h5: [[29406.354]]
Prediction for P_COSS.h5: [[29264.398]]
Prediction for P_D.h5: [[11146.8125]]
Prediction for P_G.h5: [[26372.006]]
Prediction for P_L_DCR.h5: [[0.]]
Prediction for P_ON_H_P_ON_L.h5: [[183498.9    38517.406]]
Prediction for voltage.h5: [[0.]]


In [20]:
print(175.2e-9-1.752e-07)
min = 175.2e-9
max = 262.8e-9
# 175.2e-9, 262.8e-9

print((min-min)/(max-min))
# (X[:, 4] - d_cycle_min) / (d_cycle_max - d_cycle_min)

0.0
0.0


average_current	delta_current	delta_current_percentage	average_voltage	delta_voltage	delta_voltage_percentage	DCR	ESR	P_ON_H	P_ON_L	P_COSS	P_L_DCR	P_D	P_G	P_CAP_ESR
3.345516931	10.35315758	309.4636134	2.529822269	2.600716263	102.802331	2.7032544	0.045655201	0.003303416	0.059374492	0.0124416	30.25613037	0.008205884	0.068	0.407806989


In [23]:
L_min, L_max = 30e-6, 2000e-6
C_min, C_max = 20e-6, 1000e-6
fsw_min, fsw_max = 20e3, 200e3
t_dt_min, t_dt_max = 175.2e-9, 262.8e-9
d_cycle_min, d_cycle_max = 0.2, 0.35

def custom_scaling(X):
    return np.array([
        (X[:, 0] - L_min) / (L_max - L_min), 
        (X[:, 1] - C_min) / (C_max - C_min),
        (X[:, 2] - fsw_min) / (fsw_max - fsw_min),
        (X[:, 3] - t_dt_min) / (t_dt_max - t_dt_min),
        (X[:, 4] - d_cycle_min) / (d_cycle_max - d_cycle_min)
    ]).T

# Input values provided
input_array = np.array([[30e-6, 20e-6, 20000, 175.2e-09, 0.2]])

# Perform the scaling
scaled_array = custom_scaling(input_array)

print(scaled_array)

[[0. 0. 0. 0. 0.]]


In [26]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define the min and max values for each feature as arrays
min_values = np.array([30e-6, 20e-6, 20e3, 175.2e-9, 0.2]).reshape(1, -1)
max_values = np.array([2000e-6, 1000e-6, 200e3, 262.8e-9, 0.35]).reshape(1, -1)

# Create the scaler
scaler = MinMaxScaler()

# Fit the scaler with min and max values
scaler.fit(np.vstack([min_values, max_values]))

# Define input array
input_array = np.array([[30e-6, 20e-6, 20000, 1.752e-07, 0.2]])

# Scale the input array
scaled_array = scaler.transform(input_array)

print(f"Scaled array:\n{scaled_array}")


Scaled array:
[[0. 0. 0. 0. 0.]]
